In [93]:
import warnings
warnings.filterwarnings('ignore')

from hdfs import InsecureClient
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.conf import SparkConf
from functools import reduce
from elasticsearch import Elasticsearch,helpers

import pandas as pd
import numpy as np

# Initializing Spark Session object

In [36]:
class Config:

    def __init__(self,
                elasticsearch_host,
                elasticsearch_auth_user,
                elasticsearch_auth_pass,
                hdfs_datanode_host,
                hdfs_user):
        self.elasticsearch_host = elasticsearch_host
        self.elasticsearch_auth_user = elasticsearch_auth_user
        self.elasticsearch_auth_pass = elasticsearch_auth_pass
        self.hdfs_datanode_host = hdfs_datanode_host
        self.hdfs_user = hdfs_user
        

    def _initialize_spark_session(self,appName,*,config):
        spark_config = pyspark.SparkConf().setAll([(key,value) for key,value in config.items()])
        spark_app = (SparkSession
                    .builder
                    .appName(appName)
                    .config(conf=spark_config)
                    .getOrCreate())
        return spark_app


    def _initialize_elasticsearch_client(self):
        return Elasticsearch(hosts=self.elasticsearch_host,
                            http_auth=(self.elasticsearch_auth_user,self.elasticsearch_auth_pass))

In [37]:
conf_options = {
    "spark.jars": "/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar",
    "spark.jars": "/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-spark-20_2.11-7.11.1.jar",
    "spark.es.nodes": "elasticsearch",
    "spark.es.port": "9200"
}

In [41]:
config = Config(elasticsearch_host="elasticsearch:9200",
            elasticsearch_auth_user="elastic",
            elasticsearch_auth_pass="elastic@1234",
            hdfs_datanode_host="hdfs://hadoop-filebeat:8020",
            hdfs_user="nifi")
spark = config._initialize_spark_session("SparkApplication",config=conf_options)
es = config._initialize_elasticsearch_client()

# Listing and merging all files in a single PySpark dataframe

In [50]:
# client = InsecureClient("http://hadoop-filebeat:50070",user="nifi")
# client.list('/dfs-data')

In [59]:
def fetch_and_merge_files(path,config):
    """
        Generator Function to fetch files stored in the hdfs file system and lazily evaluate them when 
        concatenating them
    """
    directory_path = f"{config.hdfs_datanode_host}/{path}"
    

    def get_files(path):
        nonlocal directory_path
        for file in client.list(path):
            print(f"Processing file: {file}")
            yield spark.read.format("csv").option("delimiter",",").option("header","true").load(f"{directory_path}/{file}")
    
    def unionAll(*dfs):
        return reduce(DataFrame.unionAll,dfs)
    
    return unionAll(*(file for file in get_files(path)))

In [55]:
merged_file = fetch_and_merge_files("/dfs-data",config)

hdfs://hadoop-filebeat:8020
hdfs://hadoop-filebeat:8020//dfs-data
Processing file: 0018ff9a-6e08-4dfc-b5dd-8d176a98cdef
Processing file: 0070cba2-e380-47d8-be13-c860a36e945b
Processing file: 00ac6b9c-2517-46c8-995b-6450d9ae4924
Processing file: 013611a0-6f1b-4429-a01e-a71af83a63aa
Processing file: 0194a7ee-e24b-4d1d-b340-6935eee4e64f
Processing file: 01f50338-69ba-4368-9af5-a887eda3a08f
Processing file: 029217cc-ef3a-4f00-9355-c93da4793ddd
Processing file: 04720985-a7bf-48a2-aa37-5d91fc85727c
Processing file: 0528f3ce-1658-47ed-97b8-c6d61aa538c5
Processing file: 05c22fdf-c4a8-4462-aaa3-f65a5649e50b
Processing file: 070d9569-47e0-4a69-8561-6eb1b0111538
Processing file: 091b4c40-00c3-479b-ad94-7f7edb2e4f99
Processing file: 09a2140c-8961-49ec-9798-488f59d1b61a
Processing file: 0be75259-a312-4a74-8a25-d64dac1396ec
Processing file: 0c6a01e6-fd74-445a-8a2f-eda91d367f73
Processing file: 0db36231-8238-4585-bf2e-199b35ac25dd
Processing file: 0fc82d5b-b035-4488-b75c-fbb4bac0f66c
Processing file:

In [86]:
df = merged_file.toPandas()
final_df = df.drop_duplicates()

# Performing Analysis

In [87]:
final_df.columns

Index(['year', 'leading_cause', 'sex', 'race_ethnicity', 'deaths',
       'death_rate', 'age_adjusted_death_rate'],
      dtype='object')

In [96]:
#replacing values 
final_df.replace([".","M","F","Accidents Except Drug Posioning (V01-X39, X43, X45-X59, Y85-Y86)","Black Non-Hispanic","White Non-Hispanic"],
                 [np.nan,"Male","Female","Accidents Except Drug Poisoning (V01-X39, X43, X45-X59, Y85-Y86)","Non-Hispanic Black","Non-Hispanic White"],
                 inplace=True)

In [97]:
def convert_columns(data_df,columns,fill_anchor_column):
    """
        converting death rates and age adjusted death rates to float types 
        and replacing the nan values with the mean of the group for that year
        
        Input
        
        data_df(pandas.DataFrame): dataframe to convert columns of
        columns(dict): dict containing column names and the type the column to be converted to
        fill_anchor_column(str): name of the anchor colume, by which the dataframe would be grouped 
                            and the filling value would be calculated
        
        Output
        
        copied_df(pandas.DataFrame): processed dataframe
    """
    copied_df = data_df.copy(deep=True)
    for column in columns:
        copied_df[column] = copied_df[column].astype(columns[column])
        copied_df[column] = copied_df.groupby(fill_anchor_column)[column].transform(lambda x:x.fillna(x.mean()))
    return copied_df


def calc_percentages(data_df,target_column):
    """
        Function to calculate percentages of each column value and output pd.Series
        
        Input:
        
        data_df(pandas.DataFrame): dataframe to calculate percentages from
        target_column(str): targt column to calculate percentages of
        
        Output:
        koalas.Series: series containing the calculated percentages for target_column of data_df
    """
    return data_df[target_column].apply(lambda x:int(x/data_df[target_column].values[0]*100))

### Some topics for analysis

#### - Analysis across genders
    - What are the leading cause of deaths per year, with their average death count, across genders
    - What are the average deaths per year, across genders
#### - Analysis for race ethnicites
    - What are the leading cause of deaths per year, across race ethnicites

Some notes:
- Death rates and age adjusted death rates are not available for minor and unknown ethnicites


In [98]:
segregated_df = final_df[~final_df['race_ethnicity'].isin(['Not Stated/Unknown','Other Race/Ethnicity'])]
segregated_df = convert_columns(segregated_df,{"deaths":float,"death_rate":float,"age_adjusted_death_rate":float},"year")

In [99]:
sex_analysis_pivot = pd.pivot_table(data=segregated_df,
                       index='year',
                       columns='sex',
                       values='deaths',
                       aggfunc=np.sum,
                       fill_value=0).astype(int)

In [100]:
sex_analysis_pivot['Female_pct_change'] = calc_percentages(sex_analysis_pivot,"Female")
sex_analysis_pivot['Male_pct_change'] = calc_percentages(sex_analysis_pivot,"Male")
sex_analysis_pivot

sex,Female,Male,Female_pct_change,Male_pct_change
year,,,,
2010,17530,7865,100,100
2011,29545,27026,168,343
2012,30134,27394,171,348
2013,28267,27212,161,345
2014,27627,25710,157,326
2015,27076,26067,154,331
2016,26769,26468,152,336
2017,26765,26553,152,337


- The above table suggests that for females, the total death counts shows a incline of 52% from 17530 to 26765, while for males the average death counts have more than tripled (surge of 237%), from 7865 to 26553. 
- This shows that the matter of concern is much higher in males as compared to females

### Lets dive deep to understand the reason for deaths amongst the genders

In [101]:
leading_cause_df = segregated_df.groupby(by=["year","sex"])[['leading_cause','deaths']].max().reset_index()
leading_cause_df[leading_cause_df['sex']=='Female']

,year,sex,leading_cause,deaths
0,2010,Female,Mental and Behavioral Disorders due to Acciden...,5351.0
2,2011,Female,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",5016.0
4,2012,Female,Septicemia (A40-A41),4719.0
6,2013,Female,Viral Hepatitis (B15-B19),4535.0
8,2014,Female,Septicemia (A40-A41),4507.0
10,2015,Female,Septicemia (A40-A41),4494.0
12,2016,Female,Septicemia (A40-A41),4306.0
14,2017,Female,Septicemia (A40-A41),4279.0


In [102]:
leading_cause_df[leading_cause_df['sex']=='Male']

,year,sex,leading_cause,deaths
1,2010,Male,Mental and Behavioral Disorders due to Acciden...,1354.0
3,2011,Male,Mental and Behavioral Disorders due to Acciden...,4220.0
5,2012,Male,Septicemia (A40-A41),4156.0
7,2013,Male,Mental and Behavioral Disorders due to Acciden...,4085.0
9,2014,Male,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",3990.0
11,2015,Male,Viral Hepatitis (B15-B19),4015.0
13,2016,Male,Peptic Ulcer (K25-K28),4054.0
15,2017,Male,Septicemia (A40-A41),4082.0


- The above analysis shows that for females, the leading cause of death in recent years has been Septicemia, a disease which can occur through urinary tract infections, pneumonia, kidney infections etc.
- As compared, the causes are quite different for men, between Mental and Behavioral disorders due to poisoning and other psychoactive substance use, viral Hepatitis, Peptic Ulcer and Septicemia

### Analysis of the reasons leading to the disease
- A quick look on the internet and we find out that Septicemia is caused by lung and kidney infection, body organs affected by smoking and alcohol consumption. Furthermore, psychoactive substances do include alcohol and nicotine(present in cigarettes).
- In addition risk factors of peptic ulcers include smoking and alcohol consumption. All these suggest that <b>alcohol consumption and smoking have been the major reasons causing deaths in the last 7 years</b>
- Also, looking at the surge of death counts, while the leading cause being the same, it is quite possible that alcohol consumption and smoking have increased in NYC in the span of 7 years

### Let's analyze the death rates in context of race ethnicites

In [103]:
race_ethnicity_pivot = pd.pivot_table(data=segregated_df,
                                   index='year',
                                   columns='race_ethnicity',
                                   values='deaths',
                                   aggfunc=np.sum,
                                   fill_value=0).astype(int)

In [104]:
race_ethnicity_pivot['Asian_Pacific_pct_change'] = calc_percentages(race_ethnicity_pivot,"Asian and Pacific Islander")
race_ethnicity_pivot['Hispanic_pct_change'] = calc_percentages(race_ethnicity_pivot,"Hispanic")
race_ethnicity_pivot['Non-Hispanic_Black_pct_change'] = calc_percentages(race_ethnicity_pivot,"Non-Hispanic Black")
race_ethnicity_pivot['Non-Hispanic_White_pct_change'] = calc_percentages(race_ethnicity_pivot,"Non-Hispanic White")
race_ethnicity_pivot['Other_Race_pct_change'] = calc_percentages(race_ethnicity_pivot,"Other Race/ Ethnicity")
race_ethnicity_pivot

race_ethnicity,Asian and Pacific Islander,Hispanic,Non-Hispanic Black,Non-Hispanic White,Other Race/ Ethnicity,Asian_Pacific_pct_change,Hispanic_pct_change,Non-Hispanic_Black_pct_change,Non-Hispanic_White_pct_change,Other_Race_pct_change
year,,,,,,,,,,
2010,1125,6387,4720,10347,2816,100,100,100,100,100
2011,3341,9391,13910,25549,4381,296,147,294,246,155
2012,3446,9420,13864,24904,5895,306,147,293,240,209
2013,3651,9672,13911,24891,3354,324,151,294,240,119
2014,3880,9687,13755,24533,1482,344,151,291,237,52
2015,4078,10182,14178,24568,137,362,159,300,237,4
2016,4252,10465,14239,24146,135,377,163,301,233,4
2017,4524,10637,14347,23679,131,402,166,303,228,4


- The above analysis shows that Asians and non-Hispanic black people are the two ethnicities where the surge in death counts is highest. The death counts in asians have more than quadrupled, from 1125 to 4524. Whereas, for non-hispanic black people, the death counts have more than tripled.

- Non-hispanic white people have accounted for the most deaths amongst all race ethnicities.

- The data for other races is inconsistent and any insights generated based on that data would be inconsistent.

### Let's do analysis based on the death rate amongst ethnicities

In [105]:
ethnicity_death_rate_pivot = segregated_df.groupby(by=["race_ethnicity","year","leading_cause"])[['death_rate']].mean().reset_index()
ethnicity_death_rate_pivot[ethnicity_death_rate_pivot['race_ethnicity']=="Asian and Pacific Islander"].groupby("year").max()

,race_ethnicity,leading_cause,death_rate
year,,,
2010,Asian and Pacific Islander,Malignant Neoplasms (Cancer: C00-C97),73.600000
2011,Asian and Pacific Islander,Malignant Neoplasms (Cancer: C00-C97),91.600000
2012,Asian and Pacific Islander,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",97.000000
2013,Asian and Pacific Islander,Viral Hepatitis (B15-B19),96.000000
2014,Asian and Pacific Islander,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",97.350000
2015,Asian and Pacific Islander,Parkinson's Disease (G20),96.383551
2016,Asian and Pacific Islander,Malignant Neoplasms (Cancer: C00-C97),100.725757
2017,Asian and Pacific Islander,Malignant Neoplasms (Cancer: C00-C97),108.319698


In [106]:
ethnicity_death_rate_pivot[ethnicity_death_rate_pivot['race_ethnicity']=="Non-Hispanic Black"].groupby("year").max()

,race_ethnicity,leading_cause,death_rate
year,,,
2010,Non-Hispanic Black,Human Immunodeficiency Virus Disease (HIV: B20...,217.700000
2011,Non-Hispanic Black,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",215.000000
2012,Non-Hispanic Black,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",221.950000
2013,Non-Hispanic Black,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",222.050000
2014,Non-Hispanic Black,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",217.950000
2015,Non-Hispanic Black,"Nephritis, Nephrotic Syndrome and Nephrisis (N...",229.962208
2016,Non-Hispanic Black,Mental and Behavioral Disorders due to Acciden...,233.237448
2017,Non-Hispanic Black,Mental and Behavioral Disorders due to Acciden...,243.860108


In [107]:
ethnicity_death_rate_pivot[ethnicity_death_rate_pivot['race_ethnicity']=="Non-Hispanic White"].groupby("year").max()

,race_ethnicity,leading_cause,death_rate
year,,,
2010,Non-Hispanic White,Malignant Neoplasms (Cancer: C00-C97),374.200000
2011,Non-Hispanic White,Mental and Behavioral Disorders due to Acciden...,335.250000
2012,Non-Hispanic White,Septicemia (A40-A41),321.500000
2013,Non-Hispanic White,Septicemia (A40-A41),311.250000
2014,Non-Hispanic White,Septicemia (A40-A41),307.550000
2015,Non-Hispanic White,Septicemia (A40-A41),308.206813
2016,Non-Hispanic White,Mental and Behavioral Disorders due to Acciden...,305.379117
2017,Non-Hispanic White,Mental and Behavioral Disorders due to Acciden...,303.908625


### Analysis of the death rates amongst ethnicities
- The above pivots show that the major reasons for deaths amongst Asians is Cancer. However, the death rate has increased by 47% from 73.6 in 2010 to 108.31 in 2017.
- For non-Hispanic Black people, Nephritis, Nephrotic Syndrome and Nephrisis have been the major reason for death rates for several years.
- <b>The death rate data for non-Hispanic White people is very representative of the overall population amongst the genders</b>, where Septicemia and mental and behavioral disorders due to overdose are major reasons for deaths
- The <b>death rate</b> for the most prevelant reason amongst <b>non-Hispanic White people</b> has actually <b>decreased</b> along the years. But the <b>actual death count has increased by 128%</b>. This suggests that there is not a single major reason driving the death counts amongst white people, but several causes that are critical

# Saving csv files to /csv-data in hdfs

listing files to save
- segregated_df
- sex_analysis_pivot
- leading_cause_df
- race_ethnicity_pivot
- ethnicity_death_rate_pivot

In [108]:
def save_dataframe_to_hdfs(data_df,target_file_name,path,config):
    """
        Function to store dataframe in hdfs
        
        Input:
        
        data_df(pandas.DataFrame): dataframe to write to hdfs
        host: url where hdfs is hosted
        port: port on which hdfs service is running
        path: the directory path to store dataframe to
    """
    sdf = spark.createDataFrame(data_df)
    sdf.write.format("csv").save(f"{config.hdfs_datanode_host}/{path}/{target_file_name}")

In [109]:
save_dataframe_to_hdfs(segregated_df,'segregated_df.csv','csv-data',config)
save_dataframe_to_hdfs(sex_analysis_pivot,'sex_analysis_pivot.csv','csv-data',config)
save_dataframe_to_hdfs(leading_cause_df,'leading_cause_df.csv','csv-data',config)
save_dataframe_to_hdfs(race_ethnicity_pivot,'race_ethnicity_pivot.csv','csv-data',config)
save_dataframe_to_hdfs(ethnicity_death_rate_pivot,'ethnicity_death_rate_pivot.csv','csv-data',config)

## Inserting documents into Elasticsearch cluster

In [110]:
def save_dataframe_to_elasticsearch(data_df,es_client,target_index_name):
    """
        Function to save pandas dataframe to elasticsearch
        
        Input:
        data_df(pandas.DataFrame): pandas DataFrame to store in elasticsearch
        es_client: elasticsearch client to store data
        target_index_name: name of index in elasticsearch cluster to put the data onto
        keys_to_use: columns name in dataframe to fetch
    """

    def doc_generator(data_df):
        data_df_iter = data_df.iterrows()
        for index, document in data_df_iter:
            yield{
                "_index":target_index_name,
                "_id": str(index),
                "_source": {key: document[key] for key in document.keys()}
            }

    helpers.bulk(es_client,doc_generator(data_df))

In [111]:
save_dataframe_to_elasticsearch(segregated_df,es,"segregated_data")
save_dataframe_to_elasticsearch(sex_analysis_pivot.reset_index(),es,"gender_based_analysis_data")
save_dataframe_to_elasticsearch(leading_cause_df,es,"leading_cause_data")
save_dataframe_to_elasticsearch(race_ethnicity_pivot.reset_index(),es,"race_ethnicity_data")
save_dataframe_to_elasticsearch(ethnicity_death_rate_pivot,es,"ethnicity_death_rate_data")